In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist


2023-05-26 15:39:09.350910: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-26 15:39:09.390649: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-26 15:39:09.391360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 15:39:09.913870: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load and preprocess the FashionMNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

The FashionMNIST dataset is loaded using the fashion_mnist.load_data() function. The dataset is then preprocessed by converting the pixel values to floating-point numbers and normalizing them to the range [0, 1].

In [3]:

# Build and train a deep learning model
model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation="relu"),   #reshape the input 
    layers.Dense(10, activation="softmax")  #reshape the output. for 10 classes in FashionMNIST
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)


Epoch 1/10


2023-05-26 15:39:11.027056: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


422/422 [==============================] - 2s 3ms/step - loss: 0.5714 - accuracy: 0.8059 - val_loss: 0.4398 - val_accuracy: 0.8435
Epoch 2/10
422/422 [==============================] - 1s 3ms/step - loss: 0.4162 - accuracy: 0.8529 - val_loss: 0.3991 - val_accuracy: 0.8583
Epoch 3/10
422/422 [==============================] - 1s 3ms/step - loss: 0.3706 - accuracy: 0.8693 - val_loss: 0.3847 - val_accuracy: 0.8615
Epoch 4/10
422/422 [==============================] - 1s 3ms/step - loss: 0.3519 - accuracy: 0.8739 - val_loss: 0.3594 - val_accuracy: 0.8677
Epoch 5/10
422/422 [==============================] - 1s 3ms/step - loss: 0.3294 - accuracy: 0.8825 - val_loss: 0.3566 - val_accuracy: 0.8703
Epoch 6/10
422/422 [==============================] - 1s 3ms/step - loss: 0.3133 - accuracy: 0.8873 - val_loss: 0.3453 - val_accuracy: 0.8748
Epoch 7/10
422/422 [==============================] - 1s 3ms/step - loss: 0.3036 - accuracy: 0.8896 - val_loss: 0.3387 - val_accuracy: 0.8802
Epoch 8/10
422/42

A sequential model is created using Keras, consisting of a flatten layer to reshape the input, a dense layer with 128 units and ReLU activation, and a dense output layer with 10 units (corresponding to the 10 classes in FashionMNIST) and softmax activation. The model is then compiled with the Adam optimizer and sparse categorical cross-entropy loss function. Finally, it is trained on the training data with a batch size of 128 and 10 epochs.

In [4]:
# Generate adversarial examples for multiple inputs
epsilon = 0.01

def generate_adversarial_examples(model, inputs, epsilon=0.01):
    # Create a copy of the inputs
    adversarial_examples = np.copy(inputs)

    # Convert inputs to TensorFlow tensors
    inputs = tf.convert_to_tensor(inputs)
    adversarial_examples = tf.convert_to_tensor(adversarial_examples)

    # Compute the gradients of the loss with respect to the inputs
    with tf.GradientTape() as tape:
        tape.watch(adversarial_examples)
        predictions = model(adversarial_examples)
        loss = keras.losses.sparse_categorical_crossentropy(y_test, predictions)

    gradients = tape.gradient(loss, adversarial_examples)

    # Apply perturbations to the inputs based on the gradients
    perturbations = epsilon * tf.sign(gradients)
    adversarial_examples += perturbations

    return adversarial_examples.numpy()

# Generate adversarial examples for multiple inputs
x_test_adv = generate_adversarial_examples(model, x_test, epsilon)


A function generate_adversarial_examples is defined to generate adversarial examples for multiple inputs. The function takes a deep learning model, the inputs, and an epsilon value (perturbation magnitude) as input. Adversarial examples are generated by applying perturbations to the inputs based on the gradients of the loss with respect to the inputs. TensorFlow's GradientTape is used to record the operations and compute the gradients. The function returns the adversarial examples as a NumPy array. The adversarial examples are generated using the generate_adversarial_examples function for the test inputs, and they are stored in the x_test_adv variable.

In [5]:


# Evaluate the success rate of the universal attack
def evaluate_success_rate(model, inputs, targets):
    # Get model predictions for the inputs
    original_predictions = np.argmax(model.predict(inputs), axis=1)
    adversarial_predictions = np.argmax(model.predict(x_test_adv), axis=1)

    # Count the successful attacks
    successful_attacks = np.sum(original_predictions != adversarial_predictions)

    # Calculate the success rate
    success_rate = successful_attacks / len(inputs)

    return success_rate

# Evaluate the success rate of the universal attack on the original model
success_rate_original_model = evaluate_success_rate(model, x_test, y_test)
print("Success rate on the original model:", success_rate_original_model)

# Create a new deep learning model with a different architecture
new_model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

new_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
new_model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

# Evaluate the success rate of the universal attack on the new model
success_rate_new_model = evaluate_success_rate(new_model, x_test, y_test)
print("Success rate on the new model:", success_rate_new_model)


313/313 [==============================] - 0s 897us/step
Success rate on the original model: 0.1323
Epoch 1/10
422/422 [==============================] - 1s 3ms/step - loss: 0.6117 - accuracy: 0.7930 - val_loss: 0.4584 - val_accuracy: 0.8422
Epoch 2/10
422/422 [==============================] - 1s 3ms/step - loss: 0.4351 - accuracy: 0.8474 - val_loss: 0.4030 - val_accuracy: 0.8530
Epoch 3/10
422/422 [==============================] - 1s 2ms/step - loss: 0.3957 - accuracy: 0.8630 - val_loss: 0.3910 - val_accuracy: 0.8605
Epoch 4/10
422/422 [==============================] - 1s 3ms/step - loss: 0.3724 - accuracy: 0.8685 - val_loss: 0.3915 - val_accuracy: 0.8610
Epoch 5/10
422/422 [==============================] - 1s 2ms/step - loss: 0.3524 - accuracy: 0.8741 - val_loss: 0.3733 - val_accuracy: 0.8652
Epoch 6/10
422/422 [==============================] - 1s 2ms/step - loss: 0.3357 - accuracy: 0.8795 - val_loss: 0.3541 - val_accuracy: 0.8738
Epoch 7/10
422/422 [============================

A function `evaluate_success_rate` is defined to evaluate the success rate of the universal attack. The function takes a model, inputs, and targets as input. It makes predictions on the original inputs and the adversarial examples generated earlier. Then, it compares the predicted labels of the original inputs with the adversarial examples to count the number of successful attacks. Finally, it calculates the success rate as the ratio of successful attacks to the total number of inputs.

The success rate of the universal attack is evaluated on the original model (`model`) and a new model (`new_model`) with a different architecture. The success rates for both models are computed using the `evaluate_success_rate` function and printed.

By comparing the success rates on different models, you can evaluate the transferability of the universal attack to other deep learning architectures. If the success rate remains high across different models, it suggests that the attack is more universal and not specific to a particular architecture.

